# Strategy details

This strategy is based on the following paper:
Revisiting (Revitalizing) Momentum

In [1]:
# Collect the list of the S&P 500 companies from Wikipedia and save it to a file
import os
import requests
import pandas as pd

# Get the list of S&P 500 companies from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
html = response.content
df = pd.read_html(html, header=0)[0]

tickers = df['Symbol'].tolist()
tickers.append('^GSPC')

In [2]:
# Load the data from yahoo finance
import os
import yfinance as yf

def load_data(symbol):

    direc = 'data/'
    os.makedirs(direc, exist_ok=True)

    file_name = os.path.join(direc, symbol + '.csv')

    if not os.path.exists(file_name):

        ticker = yf.Ticker(symbol)
        df = ticker.history(start='2005-01-01', end='2023-12-31')

        df.to_csv(file_name)

    df = pd.read_csv(file_name, index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert('US/Eastern')
    df['date'] = df.index

    if len(df) == 0:
        os.remove(file_name)
        return None

    return df


In [3]:
holder = []
ticker_with_data = []
for symbol in tickers:
    df = load_data(symbol)
    if df is not None:
        holder.append(df)
        ticker_with_data.append(symbol)

tickers = ticker_with_data[:]

print (f'Loaded data for {len(tickers)} companies')


BRK.B: No timezone found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (1d 2005-01-01 -> 2023-12-31)
GEV: Data doesn't exist for startDate = 1104555600, endDate = 1703998800
SOLV: Data doesn't exist for startDate = 1104555600, endDate = 1703998800


Loaded data for 500 companies


In [4]:
# We only need the monthly data, so we will resample the data,
# Open should be the first day of the month, Close should be the last day of the month
# High should be the maximum value of the month, Low should be the minimum value of the month
monthly_data = []
for data in holder:
    df = data.resample('M').agg({
        'date': 'first',
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

    df.set_index('date', inplace=True)

    monthly_data.append(df)

del holder

### Adding the factor and monthly return
Assuming the we will open at the monthly open price and will close at the monthly close

In [5]:
import numpy as np
# from multiprocessing import Pool

In [6]:

def process_data(df):
    
    df['monthly_log_return'] = np.log(df['Close'] / df['Close'].shift(1))

    df['12_month_cumulative_return'] = df['monthly_log_return'].rolling(11).sum().shift(1)

    df['signal_simple'] = np.exp(df['12_month_cumulative_return']) - 1

    df['signal_simple'] = df['signal_simple'].shift(1)

    df['11_month_rolling_std'] = df['monthly_log_return'].rolling(11).std().shift(1)

    df['signal_std_weighted'] = (np.exp(df['12_month_cumulative_return']) - 1)/df['11_month_rolling_std']

    df['signal_std_weighted'] = df['signal_std_weighted'].shift(1)

    return df


In [7]:
for i in range(len(monthly_data)):
    monthly_data[i] = process_data(monthly_data[i])

In [8]:
# Create a dataframe to hold monthly_data[-1]
market_data = monthly_data[-1].copy()

market_data['Short_run_market_return'] = market_data['monthly_log_return'].rolling(3).mean().shift(1)
market_data['Long_run_market_return'] = market_data['monthly_log_return'].rolling(120).mean().shift(1)
market_data['long_run_std'] = market_data['monthly_log_return'].rolling(120).std().shift(1)

delta = 0.8

# market_data.head()


In [9]:
# If the Long_run_market_return no value, then the panic_state_indicator is np.nan
market_data['panic_state_indicator'] = market_data['Long_run_market_return'].apply(lambda x: np.nan if np.isnan(x) else 0)

# If the Short_run_market_return is less than the Long_run_market_return - delta * long_run_std, then the panic_state_indicator is 1
market_data.loc[market_data['Short_run_market_return'] < market_data['Long_run_market_return'] - delta * market_data['long_run_std'], 'panic_state_indicator'] = 1

# If the Short_run_market_return is greater than the Long_run_market_return + delta * long_run_std, then the panic_state_indicator is 0
market_data.loc[market_data['Short_run_market_return'] >= market_data['Long_run_market_return'] + delta * market_data['long_run_std'], 'panic_state_indicator'] = 0

market_data['panic_state_indicator'] = market_data['panic_state_indicator'].shift(1)

# market_data.head()

In [10]:
# How many non nan values of the panic_state_indicator?
market_data.dropna(subset=['panic_state_indicator']).shape

# How many non nan values of long_run_std?
# market_data.dropna(subset=['long_run_std']).shape

# How many non nan values of Long_run_market_return?
# market_data.dropna(subset=['Long_run_market_return']).shape

(106, 14)

In [11]:

# Remove monthly_data[-1] from the list
monthly_data = monthly_data[:-1]

# Merge the market_data['panic_state_indicator'] with all the companies
for i in range(len(monthly_data)):
    monthly_data[i] = pd.merge(monthly_data[i], market_data['panic_state_indicator'], left_index=True, right_index=True)

In [17]:
# Rank the companies based on the signal_simple each month, and select the top 50 and bottom 50, and calculate the average return for each group
portfolio_return_WML = []
portfolio_return_with_panic_state = []
for j in range(96):
    rank = []
    monthly_return = []
    Panic_Indicator = 0
    for i in range(len(monthly_data)):
        if len(monthly_data[i]) < (96-j):
            continue
        rank.append(monthly_data[i].iloc[-(96-j)]['signal_simple'])
        monthly_return.append(monthly_data[i].iloc[-(96-j)]['monthly_log_return'])
        Panic_Indicator = monthly_data[i].iloc[-(96-j)]['panic_state_indicator']

    # Make a dataframe to hold the rank and monthly return
    df = pd.DataFrame({'rank': rank, 'monthly_return': monthly_return})

    # Sort the dataframe based on the rank
    df.sort_values(by='rank', inplace=True)

    # Calculate the average return for the top 50 and bottom 50
    top_50 = df.iloc[:50]['monthly_return'].mean()
    bottom_50 = df.iloc[-50:]['monthly_return'].mean()

    portfolio_return_WML.append(top_50 - bottom_50)
    portfolio_return_with_panic_state.append((top_50 - bottom_50) * (1 - Panic_Indicator)+ Panic_Indicator * (top_50 + bottom_50)/2)




In [13]:
print(len(portfolio_return_WML))
print(len(portfolio_return_with_panic_state))

96
96


In [14]:
# Calculate the cumulative return for the portfolio_return_WML and portfolio_return_with_panic_state
cumulative_return_WML = np.sum(np.array(portfolio_return_WML))
cumulative_return_with_panic_state = np.sum(np.array(portfolio_return_with_panic_state))

print(f'Cumulative return for the WML strategy: {cumulative_return_WML}')
print(f'Cumulative return for the WML strategy with panic state: {cumulative_return_with_panic_state}')

Cumulative return for the WML strategy: 0.12120034832130622
Cumulative return for the WML strategy with panic state: 0.22875313697925675


In [15]:
# Calculate the annualized return for the portfolio_return_WML and portfolio_return_with_panic_state
annualized_return_WML = (1 + cumulative_return_WML)**(12/96) - 1
annualized_return_with_panic_state = (1 + cumulative_return_with_panic_state)**(12/96) - 1

print(f'Annualized return for the WML strategy: {annualized_return_WML}')
print(f'Annualized return for the WML strategy with panic state: {annualized_return_with_panic_state}')

# Calculate the annualized volatility for the portfolio_return_WML and portfolio_return_with_panic_state
annualized_volatility_WML = np.std(portfolio_return_WML) * np.sqrt(12)
annualized_volatility_with_panic_state = np.std(portfolio_return_with_panic_state) * np.sqrt(12)

print(f'Annualized volatility for the WML strategy: {annualized_volatility_WML}')
print(f'Annualized volatility for the WML strategy with panic state: {annualized_volatility_with_panic_state}')

# Calculate the Sharpe ratio for the portfolio_return_WML and portfolio_return_with_panic_state
sharpe_ratio_WML = annualized_return_WML / annualized_volatility_WML
sharpe_ratio_with_panic_state = annualized_return_with_panic_state / annualized_volatility_with_panic_state

print(f'Sharpe ratio for the WML strategy: {sharpe_ratio_WML}')
print(f'Sharpe ratio for the WML strategy with panic state: {sharpe_ratio_with_panic_state}')

# Calculate the beta for the portfolio_return_WML and portfolio_return_with_panic_state
market_return = market_data['monthly_log_return'].iloc[-96:]
beta_WML = np.cov(portfolio_return_WML, market_return)[0][1] / np.var(market_return)
beta_with_panic_state = np.cov(portfolio_return_with_panic_state, market_return)[0][1] / np.var(market_return)

print(f'Beta for the WML strategy: {beta_WML}')
print(f'Beta for the WML strategy with panic state: {beta_with_panic_state}')

# Calculate the skewness for the portfolio_return_WML and portfolio_return_with_panic_state
skewness_WML = pd.Series(portfolio_return_WML).skew()
skewness_with_panic_state = pd.Series(portfolio_return_with_panic_state).skew()

print(f'Skewness for the WML strategy: {skewness_WML}')
print(f'Skewness for the WML strategy with panic state: {skewness_with_panic_state}')

Annualized return for the WML strategy: 0.014402715218970075
Annualized return for the WML strategy with panic state: 0.026084388311490292
Annualized volatility for the WML strategy: 0.1889173513172667
Annualized volatility for the WML strategy with panic state: 0.1971067153420117
Sharpe ratio for the WML strategy: 0.07623818097461169
Sharpe ratio for the WML strategy with panic state: 0.13233637558329608
Beta for the WML strategy: 0.48713902002107073
Beta for the WML strategy with panic state: 0.5627893140338172
Skewness for the WML strategy: 0.010462248514483767
Skewness for the WML strategy with panic state: -0.180284393888657


In [16]:
# Try different values of delta
for k in range(1, 20):
    delta = k/10
    portfolio_return_with_panic_state = []
    for j in range(96):
        rank = []
        monthly_return = []
        Panic_Indicator = 0
        for i in range(len(monthly_data)):
            if len(monthly_data[i]) < (96-j):
                continue
            rank.append(monthly_data[i].iloc[-(96-j)]['signal_simple'])
            monthly_return.append(monthly_data[i].iloc[-(96-j)]['monthly_log_return'])
            Panic_Indicator = market_data.iloc[-(96-j)]['Short_run_market_return'] < market_data.iloc[-(96-j)]['Long_run_market_return'] - delta * market_data.iloc[-(96-j)]['long_run_std']

        # Make a dataframe to hold the rank and monthly return
        df = pd.DataFrame({'rank': rank, 'monthly_return': monthly_return})

        # Sort the dataframe based on the rank
        df.sort_values(by='rank', inplace=True)

        # Calculate the average return for the top 50 and bottom 50
        top_50 = df.iloc[:50]['monthly_return'].mean()
        bottom_50 = df.iloc[-50:]['monthly_return'].mean()

        portfolio_return_with_panic_state.append((top_50 - bottom_50) * (1 - Panic_Indicator)+ Panic_Indicator * (top_50 + bottom_50)/2)

    cumulative_return_with_panic_state = np.sum(np.array(portfolio_return_with_panic_state))
    annualized_return_with_panic_state = (1 + cumulative_return_with_panic_state)**(12/96) - 1
    annualized_volatility_with_panic_state = np.std(portfolio_return_with_panic_state) * np.sqrt(12)
    sharpe_ratio_with_panic_state = annualized_return_with_panic_state / annualized_volatility_with_panic_state

    print(f'For delta = {delta}, the Sharpe ratio for the WML strategy with panic state: {sharpe_ratio_with_panic_state}')

For delta = 0.1, the Sharpe ratio for the WML strategy with panic state: 0.34584668931481444
For delta = 0.2, the Sharpe ratio for the WML strategy with panic state: 0.3327852379687461
For delta = 0.3, the Sharpe ratio for the WML strategy with panic state: 0.24959730373047245
For delta = 0.4, the Sharpe ratio for the WML strategy with panic state: 0.24529836879951372
For delta = 0.5, the Sharpe ratio for the WML strategy with panic state: 0.26143272776528587
For delta = 0.6, the Sharpe ratio for the WML strategy with panic state: 0.30026736288034933
For delta = 0.7, the Sharpe ratio for the WML strategy with panic state: 0.1881576523428597
For delta = 0.8, the Sharpe ratio for the WML strategy with panic state: 0.30813402848233973
For delta = 0.9, the Sharpe ratio for the WML strategy with panic state: 0.2582634835710585
For delta = 1.0, the Sharpe ratio for the WML strategy with panic state: 0.21962943618917255
For delta = 1.1, the Sharpe ratio for the WML strategy with panic state: 